# Make graph data

In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
def get_feature(graph_atoms):
    N = len(graph_atoms)
    for i in range(N):
        graph_atoms[i] = atom2vec[graph_atoms[i]]
    return np.array(graph_atoms)

In [3]:
def get_dist(crd1, crd2):
    crd1 = np.array(crd1)
    crd2 = np.array(crd2)
    return np.linalg.norm(crd1-crd2)

def get_adj(graph_crd):
    N = len(graph_crd)
    adj = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            if i!=j:
                adj[i][j]=get_dist(graph_crd[i],graph_crd[j])
    return adj

In [4]:
atoms_loc_data = pd.read_csv('../data/O_atom_location.csv')
atoms_loc_data['coord'] = [tuple(row[col] for col in ['x', 'y', 'z']) for _, row in atoms_loc_data.iterrows()]
atoms_loc_data = atoms_loc_data.drop('x', axis=1)
atoms_loc_data = atoms_loc_data.drop('y', axis=1)
atoms_loc_data = atoms_loc_data.drop('z', axis=1)
atom2vec = pd.read_csv('atom2vec_SVD20.csv', index_col=0).T
energy_data = list(pd.read_csv('mat_energy.csv')['energy'])

In [9]:
graph_data = []
for i in range(max(set(atoms_loc_data['num']))):
    nodes = get_feature(list(atoms_loc_data[atoms_loc_data['num']==i+1]['atom']))
    adj = get_adj(list(atoms_loc_data[atoms_loc_data['num']==i+1]['coord']))
    energy = energy_data[i]
    graph_data.append([nodes, adj, energy])
##### graph_data[idx][0] => N*D feature matrix
##### graph_data[idx][1] => N*N adjacency matrix
##### graph_data[idx][2] => 1 energy

# Grpah NN

In [9]:
##### make Weight => D*out_D matrix